In [1]:
from yelpapi import YelpAPI
import argparse
from pprint import pprint
from api_key import api_key


In [2]:
API_KEY = api_key
yelp_api = YelpAPI(API_KEY)
response = yelp_api.search_query(term='coffee', location='chicago, IL', sort_by='rating', limit=50, offset=51)
pprint(response)





{'businesses': [{'alias': 'brewpoint-coffee-elmhurst',
                 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}],
                 'coordinates': {'latitude': 41.89922, 'longitude': -87.94134},
                 'display_phone': '(224) 542-8263',
                 'distance': 21818.0477763561,
                 'id': 'u23RsNsjyRbwWqGaZ2ZZ2Q',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/DGflZvUSUJTXWoNe7y62ng/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '124 W Park Ave',
                              'address2': None,
                              'address3': '',
                              'city': 'Elmhurst',
                              'country': 'US',
                              'display_address': ['124 W Park Ave',
                                                  'Elmhurst, IL 60126'],
                              'state': 'IL',
                              'zip_code': '60126'},
          

                 'categories': [{'alias': 'breakfast_brunch',
                                 'title': 'Breakfast & Brunch'},
                                {'alias': 'coffee', 'title': 'Coffee & Tea'},
                                {'alias': 'tradamerican',
                                 'title': 'American (Traditional)'}],
                 'coordinates': {'latitude': 41.7805013,
                                 'longitude': -87.5962316},
                 'display_phone': '(773) 891-4240',
                 'distance': 15419.237757080156,
                 'id': '-Tp77Ce5OCpP1-KXUYtX9A',
                 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/ayXC_8IHewJgZ-sKVZTYRw/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '6300 S Woodlawn',
                              'address2': '',
                              'address3': '',
                              'city': 'Chicago',
                              'country': 'US',
            

In [79]:
coffee_shop_dict = {}


In [ ]:
for yelp_shop in response['businesses']:
    coffee_shop[yeld_shop['id']] = [] 

In [72]:
for yelp_shop in response['businesses']:
    print(yelp_shop['id'])
    print(yelp_shop['alias'])
    print(yelp_shop['is_closed'])
    print(yelp_shop['coordinates']['latitude'])
    print(yelp_shop['coordinates']['longitude'])
    print(yelp_shop['location']['display_address'][0]+', ' + yelp_shop['location']['display_address'][1])
    print(yelp_shop['location']['address1'])
    print(yelp_shop['location']['zip_code'])
    print(yelp_shop['price'])
    print(yelp_shop['rating'])
    print(yelp_shop['review_count'])

    

u23RsNsjyRbwWqGaZ2ZZ2Q
brewpoint-coffee-elmhurst
False
41.89922
-87.94134
124 W Park Ave, Elmhurst, IL 60126
124 W Park Ave
60126
$
4.5
181
ZMmdSK4oObGNCv50tT1OOA
evas-cafe-chicago-2
False
41.9089825
-87.6384134
1447 N Sedgwick, Chicago, IL 60610
1447 N Sedgwick
60610
$
4.5
228
IM9oq9HGTt8Q2QLQ2nevYA
the-bagelers-coffeehouse-chicago-3
False
41.926975
-87.650414
2461 N Lincoln Ave, Chicago, IL 60614
2461 N Lincoln Ave
60614
$
4.5
300
BCGezgDw9erzNM29CCJSpw
first-sip-cafe-chicago-2
False
41.97303
-87.65733
1057 W Argyle St, Chicago, IL 60640
1057 W Argyle St
60640
$
4.5
54
xMtXy7YAslpvjqaYT3KEIw
heritage-bicycles-general-store-chicago
False
41.9356745
-87.6629425
2959 N Lincoln Ave, Chicago, IL 60657
2959 N Lincoln Ave
60657
$$
4.5
276
c6OdRqy27yCotf6B5hSoww
nighthawk-chicago-chicago
False
41.967905
-87.713583
4744 N Kimball Ave, Chicago, IL 60625
4744 N Kimball Ave
60625
$$
4.5
91
1hJYOI26FjCQ-Ce6iKAvug
cafecito-chicago
False
41.882416
-87.633641
7 N Wells, Chicago, IL 60606
7 N Wells
6

KeyError: 'price'

In [15]:
# coffee_shop_dict = {}


In [4]:
%load_ext autoreload
%autoreload 1
%aimport yelp_to_dict
from yelp_to_dict import add_to_shop_dict


In [5]:
coffee_shop_dict={}
API_KEY = api_key
yelp_api = YelpAPI(API_KEY)
for next_50 in range(0,950, 51):
    response = yelp_api.search_query(term='coffee', location='chicago, IL', sort_by='rating', limit=50, offset=next_50)
    add_to_shop_dict(response, coffee_shop_dict)


black-fodder-coffee-chicago-9
did not work
the-rolling-bean-chicago-3
did not work
bartlebys-ice-cream-cakes-chicago
did not work


In [6]:
len(coffee_shop_dict)

913

In [16]:

for next_50 in range(0,950, 51):
    API_KEY = api_key
    yelp_api = YelpAPI(API_KEY)
    response = yelp_api.search_query(term='coffee', location='chicago, IL', sort_by='rating', limit=50, offset=next_50)

    for yelp_shop in response['businesses']:
        try:
            try:
                coffee_shop_dict[yelp_shop['id']] = (yelp_shop['alias'],
                                                     yelp_shop['is_closed'],
                                                    (yelp_shop['coordinates']['latitude']),
                                                    (yelp_shop['coordinates']['longitude']),
                                                    (yelp_shop['location']['display_address'][0]+', ' + yelp_shop['location']['display_address'][1]),
                                                    (yelp_shop['location']['address1']),
                                                    (yelp_shop['location']['zip_code']),
                                                    yelp_shop['price'],
                                                    yelp_shop['rating'],
                                                    yelp_shop['review_count'])
            except:
                coffee_shop_dict[yelp_shop['id']] = (yelp_shop['alias'],
                                                     yelp_shop['is_closed'],
                                                    (yelp_shop['coordinates']['latitude']),
                                                    (yelp_shop['coordinates']['longitude']),
                                                    (yelp_shop['location']['display_address'][0]+', ' + yelp_shop['location']['display_address'][1]),
                                                    (yelp_shop['location']['address1']),
                                                    (yelp_shop['location']['zip_code']),
                                                    'no_price',
                                                    yelp_shop['rating'],
                                                    yelp_shop['review_count'])
        except:
            print(yelp_shop['alias'])
            print('did not work')
            pass

black-fodder-coffee-chicago-9
did not work
the-rolling-bean-chicago-3
did not work
bartlebys-ice-cream-cakes-chicago
did not work
the-roost-food-truck-chicago
did not work


In [17]:
coffee_shop_dict.values()

dict_values([('two-hearted-queen-chicago', False, 41.9433099, -87.65928, '1201 W Roscoe St, Chicago, IL 60657', '1201 W Roscoe St', '60657', '$', 5.0, 239), ('backlot-coffee-chicago', False, 41.9533408903263, -87.7319341152907, '3982 N Avondale Ave, Chicago, IL 60641', '3982 N Avondale Ave', '60641', '$$', 5.0, 73), ('sip-of-hope-chicago-2', False, 41.92444, -87.70389, '3039 W Fullerton Ave, Chicago, IL 60647', '3039 W Fullerton Ave', '60647', '$', 5.0, 57), ('perkolator-chicago', False, 41.9531543, -87.7782196, '6032 W Irving Park Rd, Chicago, IL 60634', '6032 W Irving Park Rd', '60634', '$', 4.5, 188), ('sawada-coffee-chicago', False, 41.8836669921875, -87.6489334106445, '112 N Green St, Chicago, IL 60607', '112 N Green St', '60607', '$$', 4.5, 526), ('ipsento-chicago', False, 41.9186515808105, -87.6871032714844, '2035 N Western Ave, Chicago, IL 60647', '2035 N Western Ave', '60647', '$', 4.5, 818), ('the-avondale-coffee-club-chicago', False, 41.9391799714153, -87.6978301629424, '318

## Postgres

In [73]:
import psycopg2

In [74]:
conn = psycopg2.connect('dbname=coffee')

In [75]:
cur = conn.cursor()

In [76]:
for yelp_shop in response['businesses']:
    try:
        try:
            cur.execute("INSERT INTO coffeeshops (yelpid,yelalias,yelpisclosed,address_display,address1,addresszip,latitude,longitude,pricing,rating,reviewcount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (
                                                        yelp_shop['id'],
                                                        yelp_shop['alias'],
                                                        yelp_shop['is_closed'],
                                                        yelp_shop['location']['display_address'][0]+', ' + yelp_shop['location']['display_address'][1],
                                                        yelp_shop['location']['address1'],
                                                        yelp_shop['location']['zip_code'],
                                                        yelp_shop['coordinates']['latitude'],
                                                        yelp_shop['coordinates']['longitude'],
                                                        yelp_shop['price'],
                                                        yelp_shop['rating'],
                                                        yelp_shop['review_count']))
        except:
            cur.execute("INSERT INTO coffeeshops (yelpid,yelalias,yelpisclosed,address_display,address1,addresszip,latitude,longitude,rating,reviewcount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (
                                                        yelp_shop['id'],
                                                        yelp_shop['alias'],
                                                        yelp_shop['is_closed'],
                                                        yelp_shop['location']['display_address'][0]+', ' + yelp_shop['location']['display_address'][1],
                                                        yelp_shop['location']['address1'],
                                                        yelp_shop['location']['zip_code'],
                                                        yelp_shop['coordinates']['latitude'],
                                                        yelp_shop['coordinates']['longitude'],
                                                        yelp_shop['rating'],
                                                        yelp_shop['review_count']))
    except:
        pass



In [77]:
# conn.rollback()

In [78]:
cur.execute("SELECT * FROM coffeeshops;")


InternalError: current transaction is aborted, commands ignored until end of transaction block


In [62]:
cur.fetchone()

('f5p7scO1Q9UdaGbYe9bjFg',
 'two-hearted-queen-chicago',
 False,
 '1201 W Roscoe St, Chicago, IL 60657',
 '1201 W Roscoe St',
 '60657',
 Decimal('41.9433099000000'),
 Decimal('-87.6592800000000'),
 '$',
 None,
 239,
 5)

In [63]:
conn.commit()

In [69]:
cur.close()

In [70]:
conn.close()